![Image of MLDO](https://ibm.box.com/shared/static/ammgpiy7zqfuhdm2s7hwxpjs573lupcv.png)

# 1. Obtain Data

In [37]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [38]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [39]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-b8ff6e5b-af95-479b-b885-a47d96142520',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': 'mim6Sliu9PXhenc-vjUMrBezDu7PX0MTqpiW-MjgJHUM'
}

configuration_name = 'os_b1267f57e7aa4cd1b558205e90b442cf_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('historical.csv', 'predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru'))
data.take(5)


[Row(life='15', Day-01='88', Day-02='104', Day-03='126', Day-04='94', Day-05='103', Day-06='107', Day-07='126', Day-08='95', Day-09='105', Day-10='120', Day-11='88', Day-12='114', Day-13='126', Day-14='90', Day-15='118', mid='14'),
 Row(life='9', Day-01='121', Day-02='96', Day-03='101', Day-04='98', Day-05='97', Day-06='93', Day-07='132', Day-08='118', Day-09='108', Day-10='139', Day-11='115', Day-12='125', Day-13='120', Day-14='129', Day-15='119', mid='9'),
 Row(life='8', Day-01='90', Day-02='95', Day-03='120', Day-04='126', Day-05='120', Day-06='113', Day-07='116', Day-08='133', Day-09='125', Day-10='125', Day-11='106', Day-12='129', Day-13='96', Day-14='99', Day-15='93', mid='4'),
 Row(life='6', Day-01='72', Day-02='60', Day-03='56', Day-04='65', Day-05='57', Day-06='64', Day-07='74', Day-08='84', Day-09='80', Day-10='84', Day-11='64', Day-12='65', Day-13='80', Day-14='70', Day-15='64', mid='6'),
 Row(life='12', Day-01='74', Day-02='67', Day-03='65', Day-04='74', Day-05='72', Day-06

# 2. Perform data exploration

In [40]:
data = data.dropna() # drop rows with missing valuesexprs = [col(column).alias(column.replace(' ', '_')) for column in data.columns]

In [41]:
data.describe()

DataFrame[summary: string, life: string, Day-01: string, Day-02: string, Day-03: string, Day-04: string, Day-05: string, Day-06: string, Day-07: string, Day-08: string, Day-09: string, Day-10: string, Day-11: string, Day-12: string, Day-13: string, Day-14: string, Day-15: string, mid: string]

In [42]:
from pyspark.sql.functions import col

In [43]:
data.show()

+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+---+
|life|Day-01|Day-02|Day-03|Day-04|Day-05|Day-06|Day-07|Day-08|Day-09|Day-10|Day-11|Day-12|Day-13|Day-14|Day-15|mid|
+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+---+
|  15|    88|   104|   126|    94|   103|   107|   126|    95|   105|   120|    88|   114|   126|    90|   118| 14|
|   9|   121|    96|   101|    98|    97|    93|   132|   118|   108|   139|   115|   125|   120|   129|   119|  9|
|   8|    90|    95|   120|   126|   120|   113|   116|   133|   125|   125|   106|   129|    96|    99|    93|  4|
|   6|    72|    60|    56|    65|    57|    64|    74|    84|    80|    84|    64|    65|    80|    70|    64|  6|
|  12|    74|    67|    65|    74|    72|    73|    80|    80|    80|    80|    80|    61|    80|    80|    80|  6|
|  14|    90|   104|   107|    99|   126|   119|   102|    86|    88|   

In [44]:

df2 = data.select("life","Day-01","Day-02","Day-03","Day-04","Day-05","Day-06","Day-07","Day-08","Day-09","Day-10","Day-11","Day-12","Day-13","Day-14","Day-15","mid")
df2.take(5)

[Row(life='15', Day-01='88', Day-02='104', Day-03='126', Day-04='94', Day-05='103', Day-06='107', Day-07='126', Day-08='95', Day-09='105', Day-10='120', Day-11='88', Day-12='114', Day-13='126', Day-14='90', Day-15='118', mid='14'),
 Row(life='9', Day-01='121', Day-02='96', Day-03='101', Day-04='98', Day-05='97', Day-06='93', Day-07='132', Day-08='118', Day-09='108', Day-10='139', Day-11='115', Day-12='125', Day-13='120', Day-14='129', Day-15='119', mid='9'),
 Row(life='8', Day-01='90', Day-02='95', Day-03='120', Day-04='126', Day-05='120', Day-06='113', Day-07='116', Day-08='133', Day-09='125', Day-10='125', Day-11='106', Day-12='129', Day-13='96', Day-14='99', Day-15='93', mid='4'),
 Row(life='6', Day-01='72', Day-02='60', Day-03='56', Day-04='65', Day-05='57', Day-06='64', Day-07='74', Day-08='84', Day-09='80', Day-10='84', Day-11='64', Day-12='65', Day-13='80', Day-14='70', Day-15='64', mid='6'),
 Row(life='12', Day-01='74', Day-02='67', Day-03='65', Day-04='74', Day-05='72', Day-06

In [45]:


from pyspark.sql.types import FloatType
df2 = df2.withColumn("life", df2["life"].cast(FloatType()))
df2 = df2.withColumn("Day-01", df2["Day-01"].cast(FloatType()))
df2 = df2.withColumn("Day-02", df2["Day-02"].cast(FloatType()))
df2 = df2.withColumn("Day-03", df2["Day-03"].cast(FloatType()))
df2 = df2.withColumn("Day-04", df2["Day-04"].cast(FloatType()))
df2 = df2.withColumn("Day-05", df2["Day-05"].cast(FloatType()))
df2 = df2.withColumn("Day-06", df2["Day-06"].cast(FloatType()))
df2 = df2.withColumn("Day-07", df2["Day-07"].cast(FloatType()))
df2 = df2.withColumn("Day-08", df2["Day-08"].cast(FloatType()))
df2 = df2.withColumn("Day-09", df2["Day-09"].cast(FloatType()))
df2 = df2.withColumn("Day-10", df2["Day-10"].cast(FloatType()))
df2 = df2.withColumn("Day-11", df2["Day-11"].cast(FloatType()))
df2 = df2.withColumn("Day-12", df2["Day-12"].cast(FloatType()))
df2 = df2.withColumn("Day-13", df2["Day-13"].cast(FloatType()))
df2 = df2.withColumn("Day-14", df2["Day-14"].cast(FloatType()))
df2 = df2.withColumn("Day-15", df2["Day-15"].cast(FloatType()))
df2 = df2.withColumn("mid", df2["mid"].cast(FloatType()))

In [46]:
#df2 = df2.select("fDay1","fDay2","fDay3","fDay4","fDay5","fDay6","fDay7","fDay8","fDay9","fDay10","fDay11","fDay12","fDay13","fDay14","fDay15","fmid")
df2.take(5)
features=["life","Day-01","Day-02","Day-03","Day-04","Day-05","Day-06","Day-07","Day-08","Day-09","Day-10","Day-11","Day-12","Day-13","Day-14","Day-15"]

In [47]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
#stages=[]
#numericCols = df2
#label_stringIdx = StringIndexer(inputCol = 'mid', outputCol = 'label')
#assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
#stages += [assembler]

In [48]:
from pyspark.ml.feature import VectorAssembler

In [49]:

lr_data = df2.select(col("mid").alias("label"), *features )
lr_data.printSchema()

root
 |-- label: float (nullable = true)
 |-- life: float (nullable = true)
 |-- Day-01: float (nullable = true)
 |-- Day-02: float (nullable = true)
 |-- Day-03: float (nullable = true)
 |-- Day-04: float (nullable = true)
 |-- Day-05: float (nullable = true)
 |-- Day-06: float (nullable = true)
 |-- Day-07: float (nullable = true)
 |-- Day-08: float (nullable = true)
 |-- Day-09: float (nullable = true)
 |-- Day-10: float (nullable = true)
 |-- Day-11: float (nullable = true)
 |-- Day-12: float (nullable = true)
 |-- Day-13: float (nullable = true)
 |-- Day-14: float (nullable = true)
 |-- Day-15: float (nullable = true)



# 3. Train data

In [50]:
#split data to train/test

In [51]:
(training, test) = lr_data.randomSplit([.7, .3])

In [52]:
#creating ML pipeline 

In [53]:
from pyspark.ml.regression import LinearRegression

vectorAssembler = VectorAssembler(inputCols=features, outputCol="features")

lr = LinearRegression(maxIter=10, regParam=.01)

stages = [vectorAssembler, lr]
pipeline = Pipeline(stages=stages)

In [54]:
model = pipeline.fit(training)
prediction = model.transform(test)

In [55]:
prediction.show()

+-----+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------------------+------------------+
|label|life|Day-01|Day-02|Day-03|Day-04|Day-05|Day-06|Day-07|Day-08|Day-09|Day-10|Day-11|Day-12|Day-13|Day-14|Day-15|            features|        prediction|
+-----+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------------------+------------------+
|  3.0| 5.0|  59.0|  53.0|  57.0|  59.0|  52.0|  57.0|  48.0|  67.0|  58.0|  61.0|  61.0|  49.0|  69.0|  57.0|  51.0|[5.0,59.0,53.0,57...| 3.785143704167525|
|  3.0| 5.0|  60.0|  80.0|  70.0|  61.0|  58.0|  81.0|  71.0|  79.0|  79.0|  67.0|  64.0|  71.0|  75.0|  63.0|  73.0|[5.0,60.0,80.0,70...| 4.036253128585739|
|  3.0| 5.0|  75.0|  63.0|  73.0|  65.0|  66.0|  67.0|  61.0|  69.0|  66.0|  72.0|  60.0|  70.0|  53.0|  74.0|  66.0|[5.0,75.0,63.0,73...|3.9762500636630413|
|  3.0| 5.0|  88.0|  70.0|  76.0|  85.0|  88.0|  73.

In [56]:
model.transform(test)\
    .select("label", "prediction")\
    .show()

+-----+------------------+
|label|        prediction|
+-----+------------------+
|  3.0| 3.785143704167525|
|  3.0| 4.036253128585739|
|  3.0|3.9762500636630413|
|  3.0| 3.779809000109453|
|  3.0|4.3634563127335415|
|  3.0| 4.206385744621441|
|  3.0|3.9923835462743487|
|  3.0|  3.76094554521762|
|  3.0|4.0342000591062455|
|  3.0|3.7867961358705493|
|  3.0|3.8347257339227445|
|  3.0| 4.702101097639986|
|  3.0| 4.748275180706798|
|  3.0| 4.857360539570056|
|  3.0| 4.798020522600615|
|  3.0| 4.800444687859868|
|  3.0| 4.629550896084612|
|  3.0|4.6987343090931795|
|  3.0|4.8695142765094825|
|  3.0| 4.386773898146055|
+-----+------------------+
only showing top 20 rows



In [57]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(prediction)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(prediction, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(prediction, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(prediction, {eval.metricName: "r2"})
print("r2: %.3f" %r2)


RMSE: 1.707
MSE: 2.913
MAE: 1.434
r2: 0.667


# 4. Deploy Predictive Model

In [58]:
!pip install watson-machine-learning-client --upgrade

  Using cached https://files.pythonhosted.org/packages/b6/47/32daed1eb2169e93108480d7524fffa533f2375d8729d597c4b50cc2a6e8/watson_machine_learning_client-1.0.376-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/b1/02eebed49c754b01b17de7705caa8c4ceecfb4f926cdafc220c863584360/lomond-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/18/b0/8146a4f8dd402f60744fa380bc73ca47303cccf8b9190fd16a827281eac2/certifi-2019.9.11-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e0/da/55f51ea951e1b7c63a579c09dd7db825bb730ec1fe9c0180fc77bfb31448/urllib3-1.25.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b9/08/8505f192efc72bfafec79655e1d8351d219e2b80b0dec4ae71f50934c17a/tqdm-4.38.0-py2.py3-none-any.whl
  Using cached https://

In [59]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [60]:
# @hidden_cell
wml_credentials={
  "apikey": "eSzp_h439W_Cz_2-5hCXXds-o44v9O2MW2oEdMxdAXaP",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/7f92ce1185a3460579ce2c76a03b1a67:ce211bae-2d5c-4278-8eee-3b061501d712::",
  "iam_apikey_name": "auto-generated-apikey-a6f111a1-d5a7-4c09-8623-8bef9ef9031b",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/7f92ce1185a3460579ce2c76a03b1a67::serviceid:ServiceId-6b9acef4-6266-4b24-b575-98dd6e6f6940",
  "instance_id": "ce211bae-2d5c-4278-8eee-3b061501d712",
  "password": "ebec9040-87d8-44fe-b409-d71d16d4e822",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "a6f111a1-d5a7-4c09-8623-8bef9ef9031b"
}

In [61]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [62]:
model_props = {wml_client.repository.ModelMetaNames.AUTHOR_NAME: "Nerav Doshi",  
               wml_client.repository.ModelMetaNames.NAME: "Predictive Maintenance Model"}

published_model = wml_client.repository.store_model(model=model, pipeline=pipeline, meta_props=model_props, training_data=training)

In [63]:
import json
published_model_uid = wml_client.repository.get_model_uid(published_model)
model_details = wml_client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "guid": "a91052cd-c26d-45d8-ad27-93d648b67983",
    "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/ce211bae-2d5c-4278-8eee-3b061501d712/published_models/a91052cd-c26d-45d8-ad27-93d648b67983",
    "created_at": "2019-11-10T02:38:46.784Z",
    "modified_at": "2019-11-10T02:38:46.901Z"
  },
  "entity": {
    "runtime_environment": "spark-2.3",
    "learning_configuration_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/ce211bae-2d5c-4278-8eee-3b061501d712/published_models/a91052cd-c26d-45d8-ad27-93d648b67983/learning_configuration",
    "author": {
      "name": "Nerav Doshi"
    },
    "name": "Predictive Maintenance Model",
    "label_col": "label",
    "learning_iterations_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/ce211bae-2d5c-4278-8eee-3b061501d712/published_models/a91052cd-c26d-45d8-ad27-93d648b67983/learning_iterations",
    "training_data_schema": {
      "fields": [
        {
          "metadata": {
            "modeling_

In [64]:
#List all models
wml_client.repository.list_models()

------------------------------------  ----------------------------  ------------------------  -----------------
GUID                                  NAME                          CREATED                   FRAMEWORK
a91052cd-c26d-45d8-ad27-93d648b67983  Predictive Maintenance Model  2019-11-10T02:38:46.784Z  mllib-2.3
f5058b53-c771-4529-8250-bf542b625113  Predictive Maintenance Model  2019-11-10T02:36:51.418Z  mllib-2.3
7a1a5b3c-5129-448c-aebb-04cbabf05b9f  kidney                        2019-02-14T22:40:35.983Z  spss-modeler-18.1
ed2a90e7-95e6-43fe-a7ca-b97f40be6ece  demo 2                        2019-02-14T19:19:44.039Z  spss-modeler-18.1
78b13ca9-747c-4eff-af88-eb157fbe3948  ewpo                          2019-02-13T18:20:56.565Z  spss-modeler-18.1
b225e512-1335-4d88-a3b6-d381e0a3c77c  drug study 1                  2019-02-13T18:00:29.009Z  spss-modeler-18.1
9b6f03fc-e8fe-4954-b2a1-eceefac5667a  Fraud predictions_automatic   2019-02-13T17:00:55.170Z  wml-1.1
d7149ebd-60b1-4723-83a8-86

In [73]:
wml_client.repository.delete('f5058b53-c771-4529-8250-bf542b625113')

'SUCCESS'

In [66]:
created_deployment = wml_client.deployments.create(published_model_uid, name="Predictive Maintenance Model Py36 Online Deployment")



#######################################################################################

Synchronous deployment creation for uid: 'a91052cd-c26d-45d8-ad27-93d648b67983' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='6babaaee-acfd-4e96-b960-bc642851c243'
------------------------------------------------------------------------------------------------




# 5. Score and test

In [67]:
scoring_endpoint = wml_client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/ce211bae-2d5c-4278-8eee-3b061501d712/deployments/6babaaee-acfd-4e96-b960-bc642851c243/online


In [68]:
wml_client.deployments.list()

------------------------------------  ---------------------------------------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                                                 TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
6babaaee-acfd-4e96-b960-bc642851c243  Predictive Maintenance Model Py36 Online Deployment  online  DEPLOY_SUCCESS  2019-11-10T02:38:50.471Z  mllib-2.3          model
642fd128-9653-47f1-a580-9694868b542f  Predictive Maintenance Model Py36 Online Deployment  online  DEPLOY_SUCCESS  2019-11-10T02:36:56.269Z  mllib-2.3          model
f49270ba-2377-40dc-93b4-d7fb0b1bdeac  PM PredictiveModel                                   online  DEPLOY_SUCCESS  2019-11-07T23:37:07.236Z  mllib-2.3          model
6b488d5b-e2c6-4ffa-9cda-e64c931bb195  api2                                                 online  DEPLOY_SUCCESS  2019-02-15T00:25:31.594Z  spss-modeler-

In [69]:
#wml_client.deployments.delete('3cffe81c-438f-4a3c-ae0a-c13c79b1760b')

## Test model with a REST API call (we will need this for future steps in the notebooks)

This step demonstrates an "internal REST API" call to test the model (for an unpublished model). Notice that we are using internal values for the model endpoint and token. See documentation for external REST call syntax. An external REST call will have a different end point and will require authentication. 

In [70]:
scoring_payload = {
    "fields": [
    "life",
    "Day-01",
    "Day-02",
    "Day-03",
    "Day-04",
    "Day-05",
    "Day-06",
    "Day-07",
    "Day-08",
    "Day-09",
    "Day-10",
    "Day-11",
    "Day-12",
    "Day-13",
    "Day-14",
    "Day-15",
    ],
    "values": [ [5,60,80,70,61,56,78,89,67,89,45,35,80,78,98,73] ]
} 

In [71]:
predictions = wml_client.deployments.score(scoring_endpoint, scoring_payload)

In [72]:
print ('Prediction = {}'.format(predictions))

Prediction = {'fields': ['life', 'Day-01', 'Day-02', 'Day-03', 'Day-04', 'Day-05', 'Day-06', 'Day-07', 'Day-08', 'Day-09', 'Day-10', 'Day-11', 'Day-12', 'Day-13', 'Day-14', 'Day-15', 'features', 'prediction'], 'values': [[5.0, 60.0, 80.0, 70.0, 61.0, 56.0, 78.0, 89.0, 67.0, 89.0, 45.0, 35.0, 80.0, 78.0, 98.0, 73.0, [5.0, 60.0, 80.0, 70.0, 61.0, 56.0, 78.0, 89.0, 67.0, 89.0, 45.0, 35.0, 80.0, 78.0, 98.0, 73.0], 4.246972997490067]]}
